# Create Sales Assistant Prompt Template Deployment

This notebook shows how to create and deploy a Prompt Template for the Sales Assistant that can be used with the chat application.

## 1. Install Dependencies

In [ ]:
!pip install -U ibm-watsonx-ai | tail -n 1

## 2. Configure Credentials

In [ ]:
import getpass
import os
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models.prompts import PromptTemplateManager, PromptTemplate
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai import APIClient

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your watsonx.ai API key: "),
)

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id: ")

## 3. Create Sales Assistant Prompt Template

This template includes:
- System instruction for the Sales Assistant role
- Conversation history variable
- User question variable

In [ ]:
# Initialize Prompt Template Manager
prompt_mgr = PromptTemplateManager(
    credentials=credentials,
    project_id=project_id
)

# Define the Sales Assistant prompt template
sales_assistant_template = PromptTemplate(
    name="Sales Assistant - Deutsche Telekom",
    model_id="mistralai/mistral-small-3-1-24b-instruct-2503",  # You can change this
    model_params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MAX_NEW_TOKENS: 1000,
        GenParams.TEMPERATURE: 0.7,
        GenParams.STOP_SEQUENCES: ["<|user|>", "\n\nUser:", "\nUser:"]
    },
    description="Sales assistant for Deutsche Telekom products and services",
    task_ids=["generation"],
    input_variables=["conversation_history", "user_question"],
    instruction="""You are a sales assistant for Deutsche Telekom. Help the customer by answering their specific question about products and services. Only respond as the sales assistant - do not continue the conversation on behalf of the customer.

Previous conversation:
{conversation_history}
""",
    input_prefix="Customer",
    output_prefix="Sales Assistant",
    input_text="{user_question}",
    examples=[
        [
            "What mobile plans do you offer?",
            "We offer a variety of mobile plans to suit different needs! Our plans include unlimited data options, family plans with shared data, and flexible prepaid options. Would you like me to provide more details about a specific type of plan?"
        ],
        [
            "Do you have internet packages for home?",
            "Yes, we have several home internet packages! We offer fiber optic connections with speeds up to 1 Gbps, as well as DSL options. Our packages often include TV and phone services as well. What speed range are you interested in?"
        ]
    ]
)

# Store the prompt template
stored_template = prompt_mgr.store_prompt(prompt_template=sales_assistant_template)
print(f"✅ Prompt Template created with ID: {stored_template.prompt_id}")

## 4. Create Deployment

In [ ]:
# Initialize API Client
client = APIClient(wml_credentials=credentials)
client.set.default_project(project_id)

# Create deployment
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Sales Assistant Deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.BASE_MODEL_ID: "mistralai/mistral-small-3-1-24b-instruct-2503"
}

deployment_details = client.deployments.create(
    artifact_id=stored_template.prompt_id,
    meta_props=meta_props
)

deployment_id = deployment_details.get("metadata", {}).get("id")
print(f"\n✅ Deployment created successfully!")
print(f"\n📋 Deployment ID: {deployment_id}")
print(f"\n⚙️  Add this to your .env file:")
print(f"PROMPT_TEMPLATE_DEPLOYMENT_ID={deployment_id}")

## 5. Test the Deployment

In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference

# Initialize model with deployment
model = ModelInference(
    deployment_id=deployment_id,
    credentials=credentials,
    project_id=project_id
)

# Test with a sample question
test_question = "What are your best mobile plans for families?"

response = model.generate_text(
    params={
        "prompt_variables": {
            "conversation_history": "No previous conversation",
            "user_question": test_question
        }
    }
)

print(f"\n🧪 Test Question: {test_question}")
print(f"\n💬 Response:\n{response}")

## 6. View All Deployments

In [ ]:
client.deployments.list()

## Next Steps

1. Copy the `PROMPT_TEMPLATE_DEPLOYMENT_ID` from above
2. Add it to your `.env` file in the chat application
3. Restart the Streamlit app
4. The app will automatically use the deployed prompt template!

### Benefits of Using Deployed Templates:
- ✅ Centralized prompt management
- ✅ Version control for prompts
- ✅ Update prompts without changing code
- ✅ Consistent behavior across applications